In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler

from keras.models import Model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate
from keras.layers import GRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras import backend as K
import re
import warnings
warnings.filterwarnings('ignore')
import gensim

import os
os.environ['OMP_NUM_THREADS'] = '4'

Using TensorFlow backend.


In [2]:
EMBEDDING_FILE = 'vocs/glove.twitter.27B.200d_w2v.txt'

In [3]:
train = pd.read_csv('pre_data/df_train_tw_corr.csv', sep=';', index_col='index')
test = pd.read_csv('pre_data/df_test_tw_corr.csv', sep=';', index_col='index')
submission = pd.read_csv('data/sample_submission.csv')

y = pd.read_csv('data/train.csv')

X_train = train["comment_text"].fillna("fillna").values
y_train = y[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values


max_features = 200000
maxlen = 460


tokenizer = text.Tokenizer(num_words=max_features, filters= '~\t\n' )
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)

In [4]:
from collections import defaultdict, Counter
import tqdm

In [ ]:
model = gensim.models.KeyedVectors.load_word2vec_format('vocs/glov')
embed_size = 200

#def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = {key:model.wv[key] for key in model.wv.vocab.keys()}

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

above_max = Counter()
in_corpus = Counter()
not_in_corpus = Counter()
for word, i in word_index.items():
    if i >= max_features: 
        above_max.update([word])
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        in_corpus.update([word])
        embedding_matrix[i] = embedding_vector
    else:
        not_in_corpus.update([word])
print('all_words {all_} \n rearly words {re} \n words in courpus {cor} \n words not in corpus {not_}'\
      .format(all_ = len(word_index),
             re = len(above_max),
             cor = len(in_corpus),
             not_ = len(not_in_corpus)))

In [5]:
model = gensim.models.KeyedVectors.load_word2vec_format('vocs/glove.twitter.27B.200d_w2v.txt')
embed_size = 200

#def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = {key:model.wv[key] for key in model.wv.vocab.keys()}

all_embs = np.stack(embeddings_index.values())
emb_mean, emb_std = all_embs.mean(), all_embs.std()

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))

above_max = Counter()
in_corpus = Counter()
not_in_corpus = Counter()
for word, i in word_index.items():
    if i >= max_features: 
        above_max.update([word])
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: 
        in_corpus.update([word])
        embedding_matrix[i] = embedding_vector
    else:
        not_in_corpus.update([word])
print('all_words {all_} \n rearly words {re} \n words in courpus {cor} \n words not in corpus {not_}'\
      .format(all_ = len(word_index),
             re = len(above_max),
             cor = len(in_corpus),
             not_ = len(not_in_corpus)))

all_words 303902 
 rearly words 103903 
 words in courpus 88812 
 words not in corpus 111187


In [6]:
back_model = gensim.models.KeyedVectors.load_word2vec_format('vocs/wiki.en.vec')

In [7]:
words = back_model.index2word

w_rank = {}
for i,word in enumerate(words):
    w_rank[word] = i
rank_w = {v: k for k, v in w_rank.items()}

In [ ]:
def get_random_neigbour(source_voc, back_voc):
    

In [59]:
w_rank['alkalinity']

81851

In [67]:
back_model.vw.rank_w[81852]

'abelardo'

In [68]:
embeddings_index['abelardo']

array([-0.38530999,  0.018672  ,  0.71012998, -0.2145    , -0.066383  ,
       -0.61518002, -0.57577002,  0.42109001, -0.46994001, -0.094342  ,
       -0.16861001, -0.44874001,  0.032981  ,  0.50312001, -0.13209   ,
        0.47503   , -0.34014001,  0.2683    , -0.15684   ,  0.37368   ,
       -0.1002    , -0.12398   , -0.10444   , -0.20232999,  0.25674999,
       -0.14805   , -0.2229    ,  0.14963   ,  0.15484001,  0.55812001,
        0.26065001,  0.27829   , -0.27996999, -0.67708999, -0.77495998,
       -0.41867   , -0.52727002, -0.26034999,  0.022363  ,  0.27932   ,
       -0.035946  , -0.55369002,  0.15404999,  0.17352   , -0.43766001,
        0.13330001,  0.21281999,  0.42534   ,  0.1119    ,  0.25398999,
        0.35267001, -0.09472   , -0.054745  ,  0.33546001,  0.20115   ,
       -0.44589001,  0.06521   , -0.69949001, -0.1301    , -0.021313  ,
        0.12640999, -0.039781  , -0.17640001,  0.027382  , -0.0087436 ,
       -0.090187  ,  0.1086    ,  0.062387  ,  0.020119  , -0.70

In [24]:
word_index['jrpg']

67493

In [27]:
print(len(not_in_corpus))
not_in_corpus.most_common(30)

111187


[('euagrus', 1),
 ('achaemend', 1),
 ('amikinoj', 1),
 ('rosewall', 1),
 ('sl.wikipedia.org', 1),
 ('meretricious', 1),
 ('phix', 1),
 ('geopolitically', 1),
 ('zhurlihee', 1),
 ('rosevelt', 1),
 ('alkalinity', 1),
 ('mediawiki_message', 1),
 ('hahahahahaahahahahahahahahahahahahahahaha', 1),
 ('aqualad', 1),
 ('perlmutter', 1),
 ('spieleerfinder', 1),
 ('teninch', 1),
 ('earlcampbell', 1),
 ('briseš', 1),
 ('mushrik', 1),
 ('uncollapse', 1),
 ('<allcaps>-nog', 1),
 ('chatelier', 1),
 ('mayeb', 1),
 ('j.mickelakos', 1),
 ('presupos', 1),
 ('avoid.gif', 1),
 ('fpcus', 1),
 ('godhood', 1),
 ('موجودیت', 1)]

In [17]:
print(len(above_max))
above_max.most_common(20)

103903


[('overgorge', 1),
 ('gomatheeswaran', 1),
 ('مذاهب', 1),
 ('warkana', 1),
 ('generalisieren', 1),
 ('masungit', 1),
 ('alagukanthavel.blogspot.com', 1),
 ('wikifuhrer', 1),
 ('neodredjenih', 1),
 ('армяшкиs', 1),
 ('baixar', 1),
 ('بیایی', 1),
 ('teveel', 1),
 ('sultanali', 1),
 ('kifejezetten', 1),
 ('چندسال', 1),
 ('rrënja', 1),
 ('devianart', 1),
 ('bambook', 1),
 ('mondragón', 1)]

In [8]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


def get_model():
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.5)(x)
    x = Bidirectional(GRU(250,return_sequences=True, recurrent_dropout=0.5), )(x)
    
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    x = Dropout(0.5)(conc)
    outp = Dense(6, activation="sigmoid")(x)
    
    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [10]:
%%time
model = get_model()


batch_size = 512
epochs = 4

file_path = "models/best_right_tockzer.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")

early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)


[X_tra, X_val, y_tra, y_val] = train_test_split(x_train, y_train, train_size=0.85, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

exp_decay = lambda init, fin, steps: (init/fin)**(1/(steps-1)) - 1
steps = int(len(X_tra)/batch_size) * epochs
lr_init, lr_fin = 0.001, 0.0001
lr_decay = exp_decay(lr_init, lr_fin, steps)
K.set_value(model.optimizer.lr, lr_init)
K.set_value(model.optimizer.decay, lr_decay)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc, check_point, early_stop], verbose=2)


y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred

Train on 135635 samples, validate on 23936 samples
Epoch 1/4

 ROC-AUC - epoch: 1 - score: 0.973696 

Epoch 00001: val_loss improved from inf to 0.04915, saving model to models/best_right_tockzer.hdf5
 - 890s - loss: 0.0945 - acc: 0.9699 - val_loss: 0.0491 - val_acc: 0.9818
Epoch 2/4

 ROC-AUC - epoch: 2 - score: 0.981144 

Epoch 00002: val_loss improved from 0.04915 to 0.04465, saving model to models/best_right_tockzer.hdf5
 - 891s - loss: 0.0508 - acc: 0.9813 - val_loss: 0.0447 - val_acc: 0.9831
Epoch 3/4

 ROC-AUC - epoch: 3 - score: 0.984075 

Epoch 00003: val_loss improved from 0.04465 to 0.04246, saving model to models/best_right_tockzer.hdf5
 - 891s - loss: 0.0453 - acc: 0.9829 - val_loss: 0.0425 - val_acc: 0.9840
Epoch 4/4

 ROC-AUC - epoch: 4 - score: 0.986374 

Epoch 00004: val_loss improved from 0.04246 to 0.04049, saving model to models/best_right_tockzer.hdf5
 - 891s - loss: 0.0420 - acc: 0.9838 - val_loss: 0.0405 - val_acc: 0.9844
CPU times: user 1h 5min 14s, sys: 12min 5

In [12]:
hist = model.fit(X_tra, y_tra, batch_size=batch_size, initial_epoch=4, epochs=8, validation_data=(X_val, y_val),
                 callbacks=[RocAuc, check_point, early_stop], verbose=2)

Train on 135635 samples, validate on 23936 samples
Epoch 5/8

 ROC-AUC - epoch: 5 - score: 0.987377 

Epoch 00005: val_loss did not improve
 - 889s - loss: 0.0387 - acc: 0.9848 - val_loss: 0.0416 - val_acc: 0.9840
Epoch 6/8

 ROC-AUC - epoch: 6 - score: 0.988581 

Epoch 00006: val_loss improved from 0.04049 to 0.03943, saving model to models/best_right_tockzer.hdf5
 - 891s - loss: 0.0368 - acc: 0.9853 - val_loss: 0.0394 - val_acc: 0.9847
Epoch 7/8

 ROC-AUC - epoch: 7 - score: 0.988643 

Epoch 00007: val_loss did not improve
 - 888s - loss: 0.0351 - acc: 0.9860 - val_loss: 0.0396 - val_acc: 0.9847
Epoch 8/8

 ROC-AUC - epoch: 8 - score: 0.988781 

Epoch 00008: val_loss did not improve
 - 888s - loss: 0.0337 - acc: 0.9865 - val_loss: 0.0404 - val_acc: 0.9848


In [14]:
model.load_weights('models/best_right_tockzer.hdf5')
y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred

In [15]:
submission.to_csv('submission_1103.csv', index=False)

In [16]:
! kg config -c jigsaw-toxic-comment-classification-challenge -u cerdgio86@gmail.com -p 68918082

! kg submit 'submission_1103.csv' -m "___s____"

0.9834


In [29]:
from keras.models import load_model
model.save('models/my_model_9829.h5')

In [ ]:
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from keras.layers import GRU, BatchNormalization, Conv1D, MaxPooling1D

file_path = "best_model.hdf5"
check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                              save_best_only = True, mode = "min")
ra_val = RocAucEvaluation(validation_data=(X_valid, Y_valid), interval = 1)
early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 5)

def build_model(lr = 0.0, lr_d = 0.0, units = 0, dr = 0.0):
    inp = Input(shape = (max_len,))
    x = Embedding(max_features, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x = SpatialDropout1D(dr)(x)

    x = Bidirectional(GRU(units, return_sequences = True))(x)
    x = Conv1D(64, kernel_size = 2, padding = "valid", kernel_initializer = "he_uniform")(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    x = concatenate([avg_pool, max_pool])

    x = Dense(6, activation = "sigmoid")(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "binary_crossentropy", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, Y_train, batch_size = 128, epochs = 4, validation_data = (X_valid, Y_valid), 
                        verbose = 1, callbacks = [ra_val, check_point, early_stop])
    model = load_model(file_path)
    return model